# Developing a toy model for exploring uncertainties in the Pearson correlation coefficient

We want a model framework that produces "climate-like" signals with the following properties:
- ability to dial up/down the serial correlation
- ability to dial up/down the number of ensemble members to produce realistic smoothing
- explore the effect of trends

In [2]:
import xarray as xr

In [3]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [4]:
DATA_DIR = "../../data/"

## Load some example real signals to try and replicate

In [7]:
precip = xr.open_zarr(
    f"{DATA_DIR}/processed/CanESM5_ctrl.annual.anom_6571-6600.precip_global.zarr"
)

sst = xr.open_zarr(
    f"{DATA_DIR}/processed/CanESM5_ctrl.annual.anom_6571-6600.sst_global.zarr"
)

In [8]:
precip

<xarray.Dataset>
Dimensions:  (lat: 90, lon: 144, time: 1051)
Coordinates:
    area     (lat, lon) float32 dask.array<chunksize=(10, 12), meta=np.ndarray>
  * lat      (lat) float64 -89.49 -87.98 -85.96 -83.93 ... 85.96 87.98 89.49
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * time     (time) object 5550-12-01 00:00:00 ... 6600-12-01 00:00:00
    zsurf    (lat, lon) float32 dask.array<chunksize=(10, 12), meta=np.ndarray>
Data variables:
    precip   (time, lat, lon) float32 dask.array<chunksize=(1051, 10, 12), meta=np.ndarray>

In [ ]:
nsamples = 100_000
ar = np.array([1, -0.24, 0.16])

arma_process = ArmaProcess(ar)
times = get_lead(hcst_region, 12 + 11).time
s = arma_process.generate_sample(nsample=nsamples * len(times))
s = xr.DataArray(
    np.reshape(s, (nsamples, len(times))),
    coords={"sample": range(nsamples), "time": times},
).to_dataset(name=var_name)

obsv = {"1-year": s.sel(time=slice(*verif_period))}

average = [2, 4, 8]
for av in average:
    obsv[f"{av}-year"] = utils.rolling_mean(s, window_size=av, dim="time").sel(
        time=slice(*verif_period)
    )